In [ ]:
# !pip install llama_index.llms.azure_openai
# !pip install llama_index.embeddings.azure_openai
# !pip install python-dotenv
# !pip install pymupdf
# !pip install azure
# !pip install azure-ai-documentintelligence
# !pip install surya-ocr
# !pip install pytesseract
# !pip install pandas
# !pip install llama-index llama-index-experimental
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

  Using cached cryptography-43.0.3-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.5 MB/s eta 0:00:004.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.7 MB/s eta 0:00:00 MB/s eta 0:00:01
Using cached cryptography-43.0.3-cp39-abi3-manylinux_2_28_x86_64.whl (4.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.1 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.0 MB/s eta 0:00:005.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 5.6 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 5.9 MB/s eta 0:00:006.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.8 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 6.0 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.9 MB/s eta 0:00:00

False



# Query Engine Flow:

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core.settings import Settings
import os

# Load documents from the 'data' folder
documents = SimpleDirectoryReader('data').load_data()

# Use Azure OpenAI as the LLM for LlamaIndex
llm_deployment = os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT")
model = "gpt-4o"
temperature = 0.7
max_tokens = 4000
embedding_model = "text-embedding-3-large"
embedding_deployment = embedding_model

llm_config = {    "api_key": api_key,    "api_version": api_version,    
                  "azure_endpoint": azure_endpoint,   
                  "deployment_name": llm_deployment,    
                  "model": model,   
                  "temperature": temperature,   
                  "max_tokens": int(max_tokens) if max_tokens is not None else None }

embedding_config = { "api_key": api_key,    
                    "api_version": api_version,    
                    "azure_endpoint": azure_endpoint,   
                    "deployment_name": embedding_deployment,    
                    "model": embedding_model,  
                    "dimensions": 1024}

Settings.embed_model = AzureOpenAIEmbedding(**embedding_config)

Settings.llm = AzureOpenAI(**llm_config)
# service_context = ServiceContext.from_defaults(llm=llm)

# Build the index
index = VectorStoreIndex.from_documents(documents)

# Persist the index for future queries
index.storage_context.persist(persist_dir="./storage")
# Query the index using Azure OpenAI
# query_engine = index.as_query_engine()
# response = query_engine.query("What grade did I get in my Machine Vision and Big Data courses?")
# print(f"Response: {response}")